# Scenario
A common problem when creating models to generate business value from data is that the datasets can be so large that it can take days for the model to generate predictions. Ensuring that your dataset is stored as efficiently as possible is crucial for allowing these models to run on a more reasonable timescale without having to reduce the size of the dataset.

You've been hired by a major online data science training provider called *Training Data Ltd.* to clean up one of their largest customer datasets. This dataset will eventually be used to predict whether their students are looking for a new job or not, information that they will then use to direct them to prospective recruiters.

You've been given access to `customer_train.csv`, which is a subset of their entire customer dataset, so you can create a proof-of-concept of a much more efficient storage solution. The dataset contains anonymized student information, and whether they were looking for a new job or not during training:

| Column                   | Description                                                                      |
|------------------------- |--------------------------------------------------------------------------------- |
| `student_id`             | A unique ID for each student.                                                    |
| `city`                   | A code for the city the student lives in.                                        |
| `city_development_index` | A scaled development index for the city.                                         |
| `gender`                 | The student's gender.                                                            |
| `relevant_experience`    | An indicator of the student's work relevant experience.                          |
| `enrolled_university`    | The type of university course enrolled in (if any).                              |
| `education_level`        | The student's education level.                                                   |
| `major_discipline`       | The educational discipline of the student.                                       |
| `experience`             | The student's total work experience (in years).                                  |
| `company_size`           | The number of employees at the student's current employer.                       |
| `company_type`           | The type of company employing the student.                                       |
| `last_new_job`           | The number of years between the student's current and previous jobs.             |
| `training_hours`         | The number of hours of training completed.                                       |
| `job_change`             | An indicator of whether the student is looking for a new job (`1`) or not (`0`). |

# Project requirements

- Columns containing categories with only two factors must be stored as Booleans (bool).
- Columns containing integers only must be stored as 32-bit integers (int32).
- Columns containing floats must be stored as 16-bit floats (float16).
- Columns containing nominal categorical data must be stored as the category data type.
- Columns containing ordinal categorical data must be stored as ordered categories, and not mapped to numerical values, with an order that reflects the natural order of the column.
- The DataFrame should be filtered to only contain students with 10 or more years of experience at companies with at least 1000 employees, as their recruiter base is suited to more experienced professionals at enterprise companies.

In [382]:
# Import necessary libraries
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
import matplotlib.pyplot as plt

# Load the dataset
ds_jobs = pd.read_csv("customer_train.csv")

# View the dataset
ds_jobs.head(10)

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
0,8949,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevant experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevant experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevant experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevant experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0
5,21651,city_176,0.764,NaN,Has relevant experience,Part time course,Graduate,STEM,11,NaN,NaN,1,24,1.0
6,28806,city_160,0.920,Male,Has relevant experience,no_enrollment,High School,NaN,5,50-99,Funded Startup,1,24,0.0
7,402,city_46,0.762,Male,Has relevant experience,no_enrollment,Graduate,STEM,13,<10,Pvt Ltd,>4,18,1.0
8,27107,city_103,0.920,Male,Has relevant experience,no_enrollment,Graduate,STEM,7,50-99,Pvt Ltd,1,46,1.0
9,699,city_103,0.920,NaN,Has relevant experience,no_enrollment,Graduate,STEM,17,10000+,Pvt Ltd,>4,123,0.0


In [383]:
# Create a copy of ds_jobs for transforming
data = ds_jobs.copy()

# EDA

I am beginning the process by simply exploring the data and getting acquainted with the dataset.

In [384]:
# Finding data types
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19158 entries, 0 to 19157
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   student_id              19158 non-null  int64  
 1   city                    19158 non-null  object 
 2   city_development_index  19158 non-null  float64
 3   gender                  14650 non-null  object 
 4   relevant_experience     19158 non-null  object 
 5   enrolled_university     18772 non-null  object 
 6   education_level         18698 non-null  object 
 7   major_discipline        16345 non-null  object 
 8   experience              19093 non-null  object 
 9   company_size            13220 non-null  object 
 10  company_type            13018 non-null  object 
 11  last_new_job            18735 non-null  object 
 12  training_hours          19158 non-null  int64  
 13  job_change              19158 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

Everything looks to be the proper dtypes. There are a couple columns with missing values. I will come back to that after I review some descriptive statistics.

In [385]:
data.describe()

,student_id,city_development_index,training_hours,job_change
count,19158.000000,19158.000000,19158.000000,19158.000000
mean,16875.358179,0.828848,65.366896,0.249348
std,9616.292592,0.123362,60.058462,0.432647
min,1.000000,0.448000,1.000000,0.000000
25%,8554.250000,0.740000,23.000000,0.000000
50%,16982.500000,0.903000,47.000000,0.000000
75%,25169.750000,0.920000,88.000000,0.000000
max,33380.000000,0.949000,336.000000,1.000000


In [386]:
data[['city', 'gender',
       'relevant_experience', 'enrolled_university', 'education_level',
       'major_discipline', 'experience', 'company_size', 'company_type',
       'last_new_job']].describe()

,city,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job
count,19158,14650,19158,18772,18698,16345,19093,13220,13018,18735
unique,123,3,2,3,5,6,22,8,6,6
top,city_103,Male,Has relevant experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,1
freq,4355,13221,13792,13817,11598,14492,3286,3083,9817,8040


Notes:
- gender column has alot of missing values, as well as 3 unuiqe values.
- experience column should be an int64, is an object because > is used. Explore that column as well.
- last_new_job column also stored as obj because of > symbol.

More EDA plan:
- Explore the gender column's multiple values
- Explore experience & last job column

In [387]:
# Getting all of the unuiqe values in the gender column
data['gender'].value_counts()

Male      13221
Female     1238
Other       191
Name: gender, dtype: int64

Just Male, Female, and Other. Will be important for categorization later on.

In [388]:
# Getting a look at all the unuiqe values in the experience column
data['experience'].value_counts()

>20    3286
5      1430
4      1403
3      1354
6      1216
2      1127
7      1028
10      985
9       980
8       802
15      686
11      664
14      586
1       549
<1      522
16      508
12      494
13      399
17      342
19      304
18      280
20      148
Name: experience, dtype: int64

Ranges from 2 to 10, and then has >20

In [389]:
# Getting a look at the last job column and all its unuiqe values
data['last_new_job'].value_counts()

1        8040
>4       3290
2        2900
never    2452
4        1029
3        1024
Name: last_new_job, dtype: int64

Ranges from 1-4, then has >4

# Converting Dtypes

Now that the data is filtered, I will change all the dtypes to the desired format then export the csv.

In [390]:
# Numerical coding the relevant experience column
data['relevant_experience'] = data['relevant_experience'].replace(to_replace = 'Has relevant experience', value = 1)

In [391]:
data['relevant_experience'] = data['relevant_experience'].replace(to_replace = 'No relevant experience', value = 0)

In [392]:
data['relevant_experience'] = data['relevant_experience'].astype('bool')

In [393]:
# Changing float to float16
data['city_development_index'] = data['city_development_index'].astype('float16')

In [394]:
# Changing dtypes to categorical
data['gender'] = data['gender'].astype('category')

In [395]:
# Assuming the order is 'Primary School', 'High School', 'Graduate', 'Masters', 'Phd'
education_level_order = CategoricalDtype(['Primary School', 'High School', 'Graduate', 'Masters', 'Phd'], ordered=True)
data['education_level'] = data['education_level'].astype(education_level_order)

In [396]:
# Assuming the order is 'no_enrollment', 'Part time course', 'Full time course'
enrolled_university_order = CategoricalDtype(['no_enrollment', 'Part time course', 'Full time course'], ordered=True)
data['enrolled_university'] = data['enrolled_university'].astype(enrolled_university_order)

In [397]:
# Changing job_change category to bool
data['job_change'] = data['job_change'].replace(to_replace=1, value='True')
data['job_change'] = data['job_change'].replace(to_replace=0, value='False')
data['job_change'] = data['job_change'].astype('bool')

In [398]:
# Changing int64 to int32
data['training_hours'] = data['training_hours'].astype('int32')
data['student_id'] = data['student_id'].astype('int32')

In [399]:
# Assuming the order is '10', '>20'
experience_order = CategoricalDtype(['10', '>20'], ordered=True)
data['experience'] = data['experience'].astype(experience_order)

In [400]:
# Assuming the order is '1000-4999', '10000+'
company_size_order = CategoricalDtype(['1000-4999', '10000+'], ordered=True)
data['company_size'] = data['company_size'].astype(company_size_order)

In [401]:
# Assuming the order is 'never', '1', '2', '3', '4', '>4'
last_job_order = CategoricalDtype(['never', '1', '2', '3', '4', '>4'], ordered = True)
data['last_new_job'] = data['last_new_job'].astype(last_job_order)

In [402]:
data['city'] = data['city'].astype('category')
data['major_discipline'] = data['major_discipline'].astype('category')
data['company_type'] = data['company_type'].astype('category')

# Filtering

In [410]:
# Filter data to meet requirements
data = data[(data['experience'] >= '10') & (data['company_size'] >= '1000-4999')]

# Review and Export

Before I export I just want to get a quick look at the data to ensure everything is in the proper format, then I will export.

In [404]:
# Renaming to ds_jobs_transformed and doing final review
ds_jobs_transformed = data
ds_jobs_transformed.head()

,student_id,city,city_development_index,gender,relevant_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,job_change
12,25619,city_61,0.913086,Male,True,no_enrollment,Graduate,STEM,>20,1000-4999,Pvt Ltd,3,23,True
108,25349,city_16,0.910156,Male,True,no_enrollment,Graduate,STEM,>20,1000-4999,Pvt Ltd,>4,28,True
115,20576,city_97,0.924805,Male,True,no_enrollment,Graduate,STEM,>20,1000-4999,Pvt Ltd,>4,19,True
130,3921,city_36,0.893066,NaN,False,no_enrollment,Phd,STEM,>20,1000-4999,Public Sector,>4,4,True
154,12154,city_16,0.910156,NaN,True,no_enrollment,Masters,Arts,>20,1000-4999,NGO,>4,55,True


In [405]:
ds_jobs_transformed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 958 entries, 12 to 19143
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype   
---  ------                  --------------  -----   
 0   student_id              958 non-null    int32   
 1   city                    958 non-null    category
 2   city_development_index  958 non-null    float16 
 3   gender                  802 non-null    category
 4   relevant_experience     958 non-null    bool    
 5   enrolled_university     949 non-null    category
 6   education_level         953 non-null    category
 7   major_discipline        915 non-null    category
 8   experience              958 non-null    category
 9   company_size            958 non-null    category
 10  company_type            937 non-null    category
 11  last_new_job            953 non-null    category
 12  training_hours          958 non-null    int32   
 13  job_change              958 non-null    bool    
dtypes: bool(2), category(9)

In [406]:
# Saving ds_jobs_transformed to a csv
ds_jobs_transformed.to_csv('ds_jobs_transformed.csv')